In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import wrds
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
import datetime
import pickle as pkl
import pyarrow.feather as feather
import multiprocessing as mp
import os

In [2]:
os.environ["PGPASSWORD"] = "Fzc200202226016"
conn = wrds.Connection(wrds_username="frankie_i")

Loading library list...
Done


## Beta

#### 提取

In [2]:
crsp = conn.raw_sql("""
                      select a.permno, a.date, a.ret, b.rf, b.mktrf, b.smb, b.hml
                      from crsp.dsf as a
                      left join ff.factors_daily as b
                      on a.date=b.date
                      where a.date > '01/01/1990'
                      """)

In [7]:
dlret = conn.raw_sql("""
                     select permno, dlret, dlstdt 
                     from crsp.dsedelist
                     """)

In [4]:
crsp.to_feather('D:/股票项目/code from CHKU/save_2/beta/crsp.feather')
dlret.to_feather('D:/股票项目/code from CHKU/save_2/beta/dlret.feather')

#### 计算

In [4]:
crsp = pd.read_feather('D:/股票项目/code from CHKU/save_2/beta/crsp.feather')
dlret = pd.read_feather('D:/股票项目/code from CHKU/save_2/beta/dlret.feather')

In [5]:
# sort variables by permno and date
crsp = crsp.sort_values(by=['permno', 'date'])

# change variable format to int
crsp['permno'] = crsp['permno'].astype(int)

# Line up date to be end of month
crsp['date'] = pd.to_datetime(crsp['date'])

dlret.permno = dlret.permno.astype(int)
dlret['dlstdt'] = pd.to_datetime(dlret['dlstdt'])
dlret['date'] = dlret['dlstdt']

# merge delisting return to crsp return
crsp = pd.merge(crsp, dlret, how='left', on=['permno', 'date'])
crsp['dlret'] = crsp['dlret'].fillna(0)
crsp['ret'] = crsp['ret'].fillna(0)
crsp['retadj'] = (1 + crsp['ret']) * (1 + crsp['dlret']) - 1
crsp['exret'] = crsp['retadj'] - crsp['rf']

# find the closest trading day to the end of the month
crsp['monthend'] = crsp['date'] + MonthEnd(0)
crsp['date_diff'] = crsp['monthend'] - crsp['date']
date_temp = crsp.groupby(['permno', 'monthend'])['date_diff'].min()
date_temp = pd.DataFrame(date_temp)  # convert Series to DataFrame
date_temp.reset_index(inplace=True)
date_temp.rename(columns={'date_diff': 'min_diff'}, inplace=True)
crsp = pd.merge(crsp, date_temp, how='left', on=['permno', 'monthend'])
crsp['sig'] = np.where(crsp['date_diff'] == crsp['min_diff'], 1, np.nan)

# label every date of month end
crsp['month_count'] = crsp[crsp['sig'] == 1].groupby(['permno']).cumcount()

# label numbers of months for a firm
month_num = crsp[crsp['sig'] == 1].groupby(['permno'])['month_count'].tail(1)
month_num = month_num.astype(int)
month_num = month_num.reset_index(drop=True)

# mark the number of each month to each day of this month
crsp['month_count'] = crsp.groupby(['permno'])['month_count'].fillna(method='bfill')

# crate a firm list
df_firm = crsp.drop_duplicates(['permno'])
df_firm = df_firm[['permno']]
df_firm['permno'] = df_firm['permno'].astype(int)
df_firm = df_firm.reset_index(drop=True)
df_firm = df_firm.reset_index()
df_firm = df_firm.rename(columns={'index': 'count'})
df_firm['month_num'] = month_num

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_18648\3551634627.py:40: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  crsp['month_count'] = crsp.groupby(['permno'])['month_count'].fillna(method='bfill')
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_18648\3551634627.py:40: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  crsp['month_count'] = crsp.groupby(['permno'])['month_count'].fillna(method='bfill')


In [6]:
def get_beta(df, firm_list):
    """

    :param df: stock dataframe
    :param firm_list: list of firms matching stock dataframe
    :return: dataframe with variance of residual
    """
    for firm, count, prog in zip(firm_list['permno'], firm_list['month_num'], range(firm_list['permno'].count()+1)):
        prog = prog + 1
        print('processing permno %s' % firm, '/', 'finished', '%.2f%%' % ((prog/firm_list['permno'].count())*100))
        for i in range(count + 1):
            # if you want to change the rolling window, please change here: i - 2 means 3 months is a window.
            temp = df[(df['permno'] == firm) & (i - 2 <= df['month_count']) & (df['month_count'] <= i)]
            # if observations in last 3 months are less 21, we drop the rvar of this month
            if temp['permno'].count() < 21:
                pass
            else:
                rolling_window = temp['permno'].count()
                index = temp.tail(1).index
                X = np.mat(temp[['mktrf']])
                Y = np.mat(temp[['exret']])
                ones = np.mat(np.ones(rolling_window)).T
                M = np.identity(rolling_window) - ones.dot((ones.T.dot(ones)).I).dot(ones.T)
                beta = (X.T.dot(M).dot(X)).I.dot((X.T.dot(M).dot(Y)))
                df.loc[index, 'beta'] = beta
    return df


def sub_df(start, end, step):
    """

    :param start: the quantile to start cutting, usually it should be 0
    :param end: the quantile to end cutting, usually it should be 1
    :param step: quantile step
    :return: a dictionary including all the 'firm_list' dataframe and 'stock data' dataframe
    """
    # we use dict to store different sub dataframe
    temp = {}
    for i, h in zip(np.arange(start, end, step), range(int((end-start)/step))):
        print('processing splitting dataframe:', round(i, 2), 'to', round(i + step, 2))
        if i == 0:  # to get the left point
            temp['firm' + str(h)] = df_firm[df_firm['count'] <= df_firm['count'].quantile(i + step)]
            temp['crsp' + str(h)] = pd.merge(crsp, temp['firm' + str(h)], how='left',
                                             on='permno').dropna(subset=['count'])
        else:
            temp['firm' + str(h)] = df_firm[(df_firm['count'].quantile(i) < df_firm['count']) & (
                    df_firm['count'] <= df_firm['count'].quantile(i + step))]
            temp['crsp' + str(h)] = pd.merge(crsp, temp['firm' + str(h)], how='left',
                                             on='permno').dropna(subset=['count'])
    return temp


def main(start, end, step):
    """

    :param start: the quantile to start cutting, usually it should be 0
    :param end: the quantile to end cutting, usually it should be 1
    :param step: quantile step
    :return: a dataframe with calculated variance of residual
    """
    df = sub_df(start, end, step)
    pool = mp.Pool()
    p_dict = {}
    for i in range(int((end-start)/step)):
        p_dict['p' + str(i)] = pool.apply_async(get_beta, (df['crsp%s' % i], df['firm%s' % i],))
    pool.close()
    pool.join()
    result = pd.DataFrame()
    print('processing pd.concat')
    for h in range(int((end-start)/step)):
        result = pd.concat([result, p_dict['p%s' % h].get()])
    return result

In [9]:
crsp

,permno,date,ret,rf,mktrf,smb,hml,dlret,dlstdt,retadj,exret,monthend,date_diff,min_diff,sig,month_count
0,10001,1990-01-02,0.0,0.00026,0.0144,-0.0068,-0.0007,0.0,NaT,0.0,-0.00026,1990-01-31,29 days,0 days,NaN,0.0
1,10001,1990-01-03,-0.012346,0.00026,-0.0006,0.0073,-0.0026,0.0,NaT,-0.012346,-0.012606,1990-01-31,28 days,0 days,NaN,0.0
2,10001,1990-01-04,0.0,0.00026,-0.0071,0.0042,-0.0022,0.0,NaT,0.0,-0.00026,1990-01-31,27 days,0 days,NaN,0.0
3,10001,1990-01-05,0.00625,0.00026,-0.0085,0.0077,-0.0021,0.0,NaT,0.00625,0.00599,1990-01-31,26 days,0 days,NaN,0.0
4,10001,1990-01-08,0.006211,0.00026,0.003,-0.0041,-0.0026,0.0,NaT,0.006211,0.005951,1990-01-31,23 days,0 days,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67623434,93436,2024-12-24,0.073572,0.00017,0.0111,-0.0009,-0.0005,0.0,NaT,0.073572,0.073402,2024-12-31,7 days,0 days,NaN,174.0
67623435,93436,2024-12-26,-0.01763,0.00017,0.0002,0.0104,-0.0019,0.0,NaT,-0.01763,-0.0178,2024-12-31,5 days,0 days,NaN,174.0
67623436,93436,2024-12-27,-0.049479,0.00017,-0.0117,-0.0066,0.0056,0.0,NaT,-0.049479,-0.049649,2024-12-31,4 days,0 days,NaN,174.0
67623437,93436,2024-12-30,-0.033012,0.00017,-0.0109,0.0012,0.0074,0.0,NaT,-0.033012,-0.033182,2024-12-31,1 days,0 days,NaN,174.0


In [ ]:
# calculate variance of residual through rolling window
# Note: please split dataframe according to your CPU situation. For example, we split dataframe to (1-0)/0.05 = 20 sub
# dataframes here, so the function will use 20 cores to calculate variance of residual.
if __name__ == '__main__':
    crsp = main(0, 1, 0.25)

# process dataframe
crsp = crsp.dropna(subset=['beta'])  # drop NA due to rolling
crsp = crsp.reset_index(drop=True)
crsp = crsp[['permno', 'date', 'beta']]

with open('D:/股票项目/code from CHKU/EquityCharacteristics-master/char60/single_char/beta.feather', 'wb') as f:
    feather.write_feather(crsp, f)

processing splitting dataframe: 0.0 to 0.25


MemoryError: Unable to allocate 516. MiB for an array with shape (67623439,) and data type int64

: 

## rvar_capm

#### 提取

In [7]:
crsp = conn.raw_sql("""
                      select a.permno, a.date, a.ret, b.rf, b.mktrf
                      from crsp.dsf as a
                      left join ff.factors_daily as b
                      on a.date=b.date
                      where a.date >= '01/01/1990'
                      """)

In [ ]:
crsp.to_feather('D:/股票项目/code from'
' CHKU/save_2/rvar_capm/crsp.feather')
dlret.to_feather('D:/股票项目/code from CHKU/save_2/rvar_capm/dlret.feather')

## rvar_ff3

#### 提取

In [3]:
crsp = conn.raw_sql("""
                      select a.permno, a.date, a.ret, b.rf, b.mktrf, b.smb, b.hml
                      from crsp.dsf as a
                      left join ff.factors_daily as b
                      on a.date=b.date
                      where a.date > '01/01/1990'
                      """)

In [8]:
crsp.to_feather('D:/股票项目/code from CHKU/save_2/rvar_ff3/crsp.feather')
dlret.to_feather('D:/股票项目/code from CHKU/save_2/rvar_ff3/dlret.feather')

## rvar_mean

#### 提取

In [9]:
crsp = conn.raw_sql("""
                    select permno, date, ret
                    from crsp.dsf
                    where date >= '01/01/1990'
                    """)

In [10]:
crsp.to_feather('D:/股票项目/code from CHKU/save_2/rvar_mean/crsp.feather')
dlret.to_feather('D:/股票项目/code from CHKU/save_2/rvar_mean/dlret.feather')

## ill

#### 提取

In [11]:
crsp = conn.raw_sql("""
                    select a.permno, a.date, a.ret, a.vol, a.prc
                    from crsp.dsf as a
                    where a.date > '01/01/1990'
                    """)

In [12]:
crsp.to_feather('D:/股票项目/code from CHKU/save_2/ill/crsp.feather')
dlret.to_feather('D:/股票项目/code from CHKU/save_2/ill/dlret.feather')

## myre（sql权限不足）

#### 提取

In [3]:
from pandasql import *

In [4]:
ibes = conn.raw_sql("""
                         select
                         ticker, statpers, meanest, fpedats, anndats_act, curr_act, fpi, medest
                         from ibes.statsum_epsus
                         where
                         /* filtering IBES */
                         statpers<ANNDATS_ACT      /*only keep summarized forecasts prior to earnings annoucement*/
                         and measure='EPS'
                         and (fpedats-statpers)>=0
                         and CURCODE='USD'
                         and fpi in ('1','2')""")

ProgrammingError: (psycopg2.errors.InsufficientPrivilege) permission denied for schema tr_ibes

[SQL: 
                         select
                         ticker, statpers, meanest, fpedats, anndats_act, curr_act, fpi, medest
                         from ibes.statsum_epsus
                         where
                         /* filtering IBES */
                         statpers<ANNDATS_ACT      /*only keep summarized forecasts prior to earnings annoucement*/
                         and measure='EPS'
                         and (fpedats-statpers)>=0
                         and CURCODE='USD'
                         and fpi in ('1','2')]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [ ]:
crsp_msf = conn.raw_sql("""
                        select permno, date, prc, cfacpr
                        from crsp.msf
                        """)

In [ ]:
ibes.to_feather('D:/股票项目/code from CHKU/save_2/myre/ibes.feather')
crsp_msf.to_feather('D:/股票项目/code from CHKU/save_2/myre/crsp_msf.feather')

## sue

#### 提取

In [3]:
comp = conn.raw_sql("""
                        select gvkey, datadate, fyearq, fqtr, epspxq, ajexq
                        from comp.fundq
                        where indfmt = 'INDL' 
                        and datafmt = 'STD'
                        and popsrc = 'D'
                        and consol = 'C'
                        and datadate >= '01/01/1990'
                        """)

In [4]:
ccm = conn.raw_sql("""
                  select gvkey, lpermno as permno, linktype, linkprim, 
                  linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where linktype in ('LU', 'LC')
                  """)

In [5]:
crsp_msf = conn.raw_sql("""
                        select distinct date
                        from crsp.msf
                        where date >= '01/01/1990'
                        """)

In [6]:
comp.to_feather('D:/股票项目/code from CHKU/save_2/sue/ibes.feather')
ccm.to_feather('D:/股票项目/code from CHKU/save_2/sue/crsp_msf.feather')
crsp_msf.to_feather('D:/股票项目/code from CHKU/save_2/sue/crsp_msf.feather')